In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data import load_categories, load_hills, load_glove_model, load_word_frequency, load_nelson

hills_csv       = load_hills('../data/hills.csv')
reba_categories = load_categories('../data/reba_categories.csv')
word_freq       = load_word_frequency('../data/word_frequency.csv')
glove_model     = load_glove_model('../data/glove/glove.6B.300d.txt')
nelson          = load_nelson(['../data/nelson_a1.csv', '../data/nelson_a2.csv'], reba_categories.keys())

Loading GloVe...


400000it [00:10, 39335.60it/s]

400000 words loaded!


In [3]:
from src.data import preprocess_hills
from src.switch import adjudicate_categorization

hills = preprocess_hills(hills_csv, glove_model, reba_categories)
reba_adj = adjudicate_categorization(reba_categories, hills)

In [4]:
from src.tree import create_animal_graph, create_norm_graph

EPSILON = 0.4
G = create_animal_graph(reba_categories.keys(), glove_model, word_freq, EPSILON)
RW = create_norm_graph(reba_categories.keys(), nelson)

## Comparing Sequence Generation Methods

In [5]:
from src.evaluate import evaluate_results
from src.search import graph_traversal, cue_switch_step, random_walk_step, sub_category_switch_step
from src.switch import get_category_transitions

transition_probs = get_category_transitions(hills)

def traversal_func(graph, sequence, current_node):
    # return sub_category_switch_step(graph, current_node, reba_categories, transition_probs)
    return cue_switch_step(graph, current_node, 1, 5)
    # return random_walk_step(graph, current_node, 0.01)

pred = []
for _ in range(100):
    pred += [graph_traversal(G, traversal_func, traversal_method='max')]
gold = [r['response'] for r in hills]

evaluate_results(pred, gold, reba_categories)

/Users/dhei/miniconda3/envs/typical/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'exemplar_bleu': 0.211679,
 'intra_category_bleu': 0.875443,
 'inter_category_bleu': {'Reptiles/Amphibians': 0,
  'Beasts of Burden': 0,
  'Arctic/Far North Animals': 0,
  'Fish': 0.608492,
  'North American': 0.139628,
  'Insects': 0.408248,
  'Canines': 0,
  'Primates': 0,
  'Australian Animals': 0,
  'Non-Jungle Animals': 0.074641,
  'Farm Animals': 0.140151,
  'Felines': 0.014286,
  'Birds': 0,
  'Rodent Like': 0,
  'Jungle Animals': 0,
  'Pets': 0.326078},
 'average_run_length': 1.56,
 'percent_switches': 0.615385}

In [6]:
from src.search import random_walk
import random

pred = []
for _ in range(100):
    pred += [random_walk(RW, start_node=random.choice(list(RW.nodes())), max_length=40, max_iter=4000)]

evaluate_results(pred, gold, reba_categories)

{'exemplar_bleu': 0.067731,
 'intra_category_bleu': 0.821119,
 'inter_category_bleu': {'Reptiles/Amphibians': 0.013865,
  'Beasts of Burden': 0,
  'Arctic/Far North Animals': 0,
  'Fish': 0.04412,
  'North American': 0.032171,
  'Insects': 0.015947,
  'Canines': 0,
  'Primates': 0.027728,
  'Australian Animals': 0,
  'Non-Jungle Animals': 0.091425,
  'Farm Animals': 0.06414,
  'Felines': 0.072705,
  'Birds': 0.008561,
  'Rodent Like': 0.041017,
  'Jungle Animals': 0,
  'Pets': 0.044277},
 'average_run_length': 2.241399,
 'percent_switches': 0.433923}

In [7]:
from src.evaluate import evaluate_results
from src.search import cue_switch_step, random_walk_step, sub_category_switch_step
from src.search import graph_traversal, MC3, beam_search
from src.switch import get_category_transitions

transition_probs = get_category_transitions(hills)

def traversal_func(graph, sequence, current_node):
    # return sub_category_switch_step(graph, current_node, reba_categories, transition_probs)
    return cue_switch_step(graph, current_node, 1, 5)
    # return random_walk_step(graph, current_node, 0.01)

pred = []
for _ in range(100):
    # pred += [graph_traversal(G, traversal_func, traversal_method='max')]
    pred += [MC3(G, traversal_func, num_chains=10)[0]]

# pred = beam_search(G, traversal_func, beam_width=10)[0]
# pred = MC3(G, traversal_func, num_chains=100)
gold = [r['response'] for r in hills]

evaluate_results(pred, gold, reba_categories)

{'exemplar_bleu': 0.017591,
 'intra_category_bleu': 0.794282,
 'inter_category_bleu': {'Reptiles/Amphibians': 0.012658,
  'Beasts of Burden': 0,
  'Arctic/Far North Animals': 0,
  'Fish': 0,
  'North American': 0.014703,
  'Insects': 0,
  'Canines': 0,
  'Primates': 0,
  'Australian Animals': 0,
  'Non-Jungle Animals': 0.019473,
  'Farm Animals': 0.074967,
  'Felines': 0.024936,
  'Birds': 0.003756,
  'Rodent Like': 0.003904,
  'Jungle Animals': 0.008367,
  'Pets': 0.042982},
 'average_run_length': 1.205488,
 'percent_switches': 0.805225}

In [8]:
import numpy as np

N_ITER = 50

def cue_func(graph, sequence, current_node):
    return cue_switch_step(graph, current_node, 0, 1)

# for t in np.arange(0.0001, 0.1, 0.01):
for t in [1e-4, 1e-2, 0.5, 1, 1.3, 1.9]:
    pred = [graph_traversal(G, cue_func, traversal_method='sample', temp=t) for _ in range(N_ITER)]
    print(f'{t:.4f}', evaluate_results(pred, gold, reba_categories)['exemplar_bleu'])

0.0001 0.163038
0.0100 0.100485
0.5000 0.033411
1.0000 0.05055
1.3000 0.059167
1.9000 0.053589


In [15]:
from src.evaluate import evaluate_results
from src.search import cue_switch_step, random_walk_step, sub_category_switch_step
from src.search import graph_traversal, MC3, beam_search
from src.switch import get_category_transitions
from src.util import render_table

transition_probs = get_category_transitions(hills)

results = {
    'Greedy': {}, 'Greedy': {}, 'Beam Search': {}, 'Sampling ($\\tau=0.3$)': {}, 
    'Sampling ($\\tau=1$)': {}, 'Sampling ($\\tau=1.5$)': {}, 'Random Walk': {}, 
    'M-H': {}, 'MC$^3$': {}
}
N_ITER = 100

gold = [r['response'] for r in hills]

def cue_func(graph, sequence, current_node):
    return cue_switch_step(graph, current_node, 1, 5)

def sub_category_func(graph, sequence, current_node):
    return sub_category_switch_step(graph, current_node, reba_categories, transition_probs)

def random_func(graph, sequence, current_node):
    return random_walk_step(graph, current_node, 0.01)

func_map = {
    'Abbott': random_func,
    'Hills': cue_func,
    'Subcategory Cue': sub_category_func
}

examples = []

for f in func_map.keys():
    # pred = [graph_traversal(G, func_map[f], traversal_method='max') for _ in range(N_ITER)]
    # examples += [pred[0]]
    # results['Greedy'][f] = evaluate_results(pred, gold, reba_categories)['exemplar_bleu']

    # pred = beam_search(G, func_map[f], beam_width=N_ITER)[0]
    # examples += [pred[0]]
    # results['Beam Search'][f] = evaluate_results(pred, gold, reba_categories)['exemplar_bleu']

    # pred = [graph_traversal(G, func_map[f], traversal_method='sample', temp=1e-2) for _ in range(N_ITER)]
    # examples += [pred[0]]
    # results['Sampling ($\\tau=0.3$)'][f] = evaluate_results(pred, gold, reba_categories)['exemplar_bleu']

    # pred = [graph_traversal(G, func_map[f], traversal_method='sample', temp=0.3) for _ in range(N_ITER)]
    # examples += [pred[0]]
    # results['Sampling ($\\tau=1$)'][f] = evaluate_results(pred, gold, reba_categories)['exemplar_bleu']

    # pred = [graph_traversal(G, func_map[f], traversal_method='sample', temp=1.7) for _ in range(N_ITER)]
    # examples += [pred[0]]
    # results['Sampling ($\\tau=1.5$)'][f] = evaluate_results(pred, gold, reba_categories)['exemplar_bleu']

    pred = [random_walk(RW, start_node=random.choice(list(RW.nodes())), max_length=40, max_iter=4000) for _ in range(N_ITER)]
    examples += [pred[0]]
    results['Random Walk'][f] = evaluate_results(pred, gold, reba_categories)['exemplar_bleu']

    # pred = [MC3(G, func_map[f], num_chains=1)[0] for _ in range(N_ITER)]
    # examples += [pred[0]]
    # results['M-H'][f] = evaluate_results(pred, gold, reba_categories)['exemplar_bleu']

    # pred = [MC3(G, func_map[f], num_chains=2)[0] for _ in range(N_ITER)]
    # examples += [pred[0]]
    # results['MC$^3$'][f] = evaluate_results(pred, gold, reba_categories)['exemplar_bleu']

print(render_table({k: v for k, v in results.items() if len(v) != 0}))

\begin{tabular}{p{3cm}C{1.5cm}C{1.5cm}C{1.5cm}}
\toprule
\textbf{Method} & \textbf{Abbott} & \textbf{Hills} & \textbf{Subcategory Cue} \\
\midrule

Random Walk & 0.08 & 0.08 & 0.07 \\

\bottomrule
\end{tabular}



In [ ]:
from src.data import REBA_CATEGORY_ABBR
from src.switch import min_category_switches

for e_idx, e in enumerate(examples[::3]):
    e = e[:50]
    cat = [REBA_CATEGORY_ABBR[c] for c in min_category_switches([reba_categories[p] for p in e])]
    run = ' '.join([f'\{cat[i]}{{{exemplar}}}' for i, exemplar in enumerate(e)]) 
    print(f'{list(results.keys())[e_idx]} & {run} \\\\')

print(f'\\midrule')

for e_idx, e in enumerate(examples[:3]):
    e = e[:50]
    cat = [REBA_CATEGORY_ABBR[c] for c in min_category_switches([reba_categories[p] for p in e])]
    run = ' '.join([f'\{cat[i]}{{{exemplar}}}' for i, exemplar in enumerate(e)]) 
    print(f'{list(func_map.keys())[e_idx]} & {run} \\\\')

Greedy & \pets{dog} \pets{cat} \primates{monkey} \nonjungle{lion} \nonjungle{tiger} \nonjungle{elephant} \farm{bear} \north{deer} \north{squirrel} \pets{mouse} \rodent{rat} \pets{snake} \pets{fish} \pets{bird} \farm{cow} \farm{horse} \farm{sheep} \farm{pig} \north{rabbit} \australian{kangaroo} \australian{wallaby} \birds{kiwi} \nonjungle{giraffe} \nonjungle{zebra} \nonjungle{leopard} \nonjungle{cheetah} \nonjungle{gazelle} \nonjungle{antelope} \north{elk} \north{moose} \north{raccoon} \north{skunk} \farm{whale} \farm{dolphin} \fish{shark} \fish{turtle} \jungle{alligator} \reptiles{lizard} \reptiles{frog} \\
Sampling ($\tau=0.3$) & \farm{ox} \farm{donkey} \farm{mule} \farm{pony} \farm{horse} \farm{bull} \nonjungle{lion} \nonjungle{elephant} \nonjungle{tiger} \fish{shark} \fish{fish} \fish{whale} \fish{turtle} \farm{snake} \pets{lizards} \pets{mammal} \reptiles{dinosaur} \reptiles{lizard} \reptiles{gecko} \nonjungle{mongoose} \insects{centipede} \pets{bulldog} \pets{terrier} \pets{dog} \

In [ ]:
from tqdm import tqdm
import statistics

# Evaluate Human Baseline
human_results = []
for i in tqdm(range(len(gold))):
    human_results += [evaluate_results([gold[i]], gold[:i] + gold[i+1:], reba_categories)]

print(statistics.mean([r['exemplar_bleu'] for r in human_results]))
print(statistics.mean([r['intra_category_bleu'] for r in human_results]))

100%|██████████| 141/141 [00:04<00:00, 30.45it/s]

0.22260202836879434
0.9001463546099291
